In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/dailymail/translated-dailymail-train.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/summarization/cnn-news/translated-cnn-train.json

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-tpu.json'

In [3]:
import numpy as np
from numpy.random import default_rng
import random
import collections
import re
import tensorflow as tf
from tqdm import tqdm
import tokenization
import json
from google.cloud import storage
import mp

max_seq_length_encoder = 1536
max_seq_length_decoder = 768

EOS_ID = 1

tokenizer = tokenization.FullTokenizer(
    vocab_file='pegasus.wordpiece', do_lower_case=False
)

In [4]:
def create_int_feature(values):
    feature = tf.train.Feature(
        int64_list=tf.train.Int64List(value=list(values))
    )
    return feature

def write_instance_to_example_file(X, Y, output_file):
    writer = tf.python_io.TFRecordWriter(output_file)
    for i in range(len(X)):
        input_ids = X[i]
        target_ids = Y[i]
        while len(input_ids) < max_seq_length_encoder:
            input_ids.append(0)
        while len(target_ids) < max_seq_length_decoder:
            target_ids.append(0)

        features = collections.OrderedDict()
        features['input_ids'] = create_int_feature(input_ids)
        features['target_ids'] = create_int_feature(target_ids)
        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features)
        )
        writer.write(tf_example.SerializeToString())

    tf.logging.info('Wrote %d total instances', inst_index)

In [5]:
def get_feature(x, y):
    tokens = tokenizer.tokenize(x)
    if len(tokens) > (max_seq_length_encoder - 2):
        tokens = tokens[: max_seq_length_encoder - 2]
    tokens = tokens

    tokens_y = []
    for y_ in y:
        tokens_y.extend(tokenizer.tokenize(y_))
    if len(tokens_y) > (max_seq_length_decoder - 1):
        tokens_y = tokens_y[: max_seq_length_decoder - 1]
    
    tokens_y = tokenizer.convert_tokens_to_ids(tokens_y)
    tokens_y = tokens_y + [EOS_ID]
    return tokenizer.convert_tokens_to_ids(tokens), tokens_y

In [6]:
def write_instance_to_example_file(X, Y, output_file):
    writer = tf.python_io.TFRecordWriter(output_file)
    for i in range(len(X)):
        input_ids = X[i]
        target_ids = Y[i]
        while len(input_ids) < max_seq_length_encoder:
            input_ids.append(0)
        while len(target_ids) < max_seq_length_decoder:
            target_ids.append(0)

        features = collections.OrderedDict()
        features['input_ids'] = create_int_feature(input_ids)
        features['target_ids'] = create_int_feature(target_ids)
        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features)
        )
        writer.write(tf_example.SerializeToString())

    tf.logging.info('Wrote %d total instances', len(X))

In [7]:
def loop(rows):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    rows, index = rows
    
    X, Y = [], []
    output_file = f'bigbird-summarization-{index}.tfrecord'
    for i in tqdm(range(len(rows))):
        tokens, tokens_y = get_feature(' '.join(data[i]['ms_article']), [' '.join(data[i]['ms_abstract'])])
        X.append(tokens)
        Y.append(tokens_y)
            
    write_instance_to_example_file(X, Y, output_file)
    blob = bucket.blob(f'bigbird-summarization-data/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [8]:
with open('translated-cnn-train.json') as fopen:
    data = json.load(fopen)
    
with open('translated-cnn-train.json') as fopen:
    data.extend(json.load(fopen))

In [9]:
import random

random.shuffle(data)

In [10]:
mp.multiprocessing(data, loop, cores = 20)

 98%|█████████▊| 8873/9057 [02:32<00:03, 57.36it/s]

INFO:tensorflow:Wrote 9057 total instances


 99%|█████████▉| 8995/9057 [02:33<00:00, 71.63it/s]

INFO:tensorflow:Wrote 9057 total instances


 99%|█████████▊| 8936/9057 [02:33<00:02, 58.85it/s]

INFO:tensorflow:Wrote 18 total instances


100%|██████████| 9057/9057 [02:35<00:00, 58.27it/s]


INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances
INFO:tensorflow:Wrote 9057 total instances


In [11]:
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.to_int32(t)
        example[name] = t

    return example

def input_fn_builder(
    input_files,
    max_seq_length_encoder,
    max_seq_length_decoder,
    max_predictions_per_seq,
    is_training,
    num_cpu_threads = 4,
):
    def input_fn(params):
        batch_size = params['batch_size']

        name_to_features = {
            'input_ids': tf.FixedLenFeature([max_seq_length_encoder], tf.int64),
            'target_ids': tf.FixedLenFeature(
                [max_seq_length_decoder], tf.int64
            ),
        }
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(input_files))
            d = d.repeat()
            d = d.shuffle(buffer_size = len(input_files))
            cycle_length = min(num_cpu_threads, len(input_files))
            d = d.apply(
                tf.contrib.data.parallel_interleave(
                    tf.data.TFRecordDataset,
                    sloppy = is_training,
                    cycle_length = cycle_length,
                )
            )
            d = d.shuffle(buffer_size = 100)
        else:
            d = tf.data.TFRecordDataset(input_files)
            d = d.repeat()
        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size = batch_size,
                num_parallel_batches = num_cpu_threads,
                drop_remainder = True,
            )
        )
        return d

    return input_fn

In [15]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/bigbird-summarization-data/*.tfrecord')
input_fn = input_fn_builder(files, 1536, 768, 0, True)
dataset = input_fn({'batch_size': 1})
dataset = dataset._make_one_shot_iterator().get_next()

In [16]:
sess = tf.Session()

In [17]:
r = sess.run(dataset)

In [18]:
import tokenization

tokenizer = tokenization.FullTokenizer(
    vocab_file='pegasus.wordpiece', do_lower_case=False
)

In [19]:
tokenizer.convert_ids_to_tokens(r['input_ids'][0])

['Nick',
 'Wil',
 '##kins',
 'didi',
 '##agnosis',
 'dengan',
 'le',
 '##uke',
 '##mi',
 '##a',
 'ketika',
 'dia',
 'berusia',
 '4',
 'tahun',
 ',',
 'dan',
 'ketika',
 'barah',
 'itu',
 'terus',
 'bangkit',
 'kembali',
 ',',
 'tidak',
 'tahan',
 'terhadap',
 'semua',
 'rawatan',
 'yang',
 'berbeza',
 'yang',
 'doktor',
 'cuba',
 ',',
 'bapanya',
 'duduk',
 'dia',
 'untuk',
 'bercakap',
 '.',
 'John',
 'Wil',
 '##kins',
 'menjelaskan',
 'kepada',
 'Nick',
 ',',
 'yang',
 'ketika',
 'itu',
 'berusia',
 '14',
 'tahun',
 ',',
 'bahawa',
 'doktor',
 'telah',
 'mencuba',
 'kem',
 '##oterapi',
 ',',
 'radiasi',
 ',',
 'malah',
 'pemindahan',
 'sum',
 '##sum',
 'tulang',
 'dari',
 'kakaknya',
 '.',
 '"',
 'I',
 'explain',
 '##ed',
 'to',
 'him',
 'that',
 'we',
 "'",
 're',
 'run',
 '##ning',
 'out',
 'of',
 'options',
 ',',
 '"',
 'Wil',
 '##kins',
 'rem',
 '##ember',
 '##s',
 'tel',
 '##ling',
 'his',
 'so',
 '##n',
 '.',
 'Terdapat',
 'satu',
 'kemungkinan',
 'rawatan',
 'yang',
 'boleh',


In [20]:
tokenizer.convert_ids_to_tokens(r['target_ids'][0])

['Nick',
 'Wil',
 '##kins',
 'was',
 'out',
 'of',
 'options',
 'for',
 'bat',
 '##t',
 '##ling',
 'le',
 '##uke',
 '##mi',
 '##a',
 '.',
 'Beliau',
 'kini',
 'bebas',
 'daripada',
 'kanser',
 'selepas',
 'menjalani',
 'rawatan',
 'eksperimen',
 '.',
 'Doktor',
 'mengajar',
 'sel',
 'imun',
 'Nick',
 'untuk',
 'menjadi',
 'mahir',
 'dalam',
 'membunuh',
 'barah',
 '.',
 'Pakar',
 'berharap',
 'rawatan',
 'itu',
 'akan',
 'cepat',
 'menjadi',
 'lebih',
 'meluas',
 '.',
 '[CLS]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]